In [3]:
import pandas as pd
import numpy as np

from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List, Optional, Tuple

import muspy

In [7]:
PROJECT_ROOT = Path.cwd()
DATA_ROOT = PROJECT_ROOT / "data" / "essen"
DATA_ROOT.mkdir(parents=True, exist_ok=True)

ds = muspy.datasets.EssenFolkSongDatabase(
    root=Path("data/essen"),
    download_and_extract=True,
    convert=False, 
    use_converted=None,
    verbose=True,
    n_jobs=-1
)

178061312it [00:01, 135520665.19it/s]                        


Successfully downloaded source : C:\Users\yaros\Desktop\ITMO\stochastic-processes\data\essen\esac.zip .
Extracting archive : C:\Users\yaros\Desktop\ITMO\stochastic-processes\data\essen\esac.zip ...
Successfully extracted archive : C:\Users\yaros\Desktop\ITMO\stochastic-processes\data\essen .


In [8]:
print("Песен:", len(ds))

Песен: 10457


In [14]:
def music_to_tokens(music: muspy.Music, step_division: int = 4):

    step_ticks = max(1, int(music.resolution) // step_division)
    tracks = [t for t in music.tracks if t.notes]
    track = max(tracks, key=lambda t: len(t.notes))

    notes = sorted(track.notes, key=lambda n: n.time)

    tokens = []
    prev_end = 0

    for n in notes:
        rest_ticks = max(0, int(n.time) - int(prev_end))

        rest = int(np.round(rest_ticks / step_ticks)) # это пауза перед нотой в шагах
        dur = max(1, int(np.round(int(n.duration) / step_ticks))) # это длительность ноты в шагах
        pitch = int(n.pitch) # это значение MIDI ноты (от 0 до 127)

        tokens.append((pitch, dur, rest))
        prev_end = max(prev_end, int(n.time) + int(n.duration)) 

    return tokens

In [19]:
import json

out_path = Path("data/essen_tokens.jsonl")
with out_path.open("w", encoding="latin-1") as f:
    kept = 0
    for music in ds:
        tokens = music_to_tokens(music, step_division=4)
        if len(tokens) < 100:
            continue
        f.write(json.dumps(tokens) + "\n")
        kept += 1

print("Сохранено песен:", kept)
print("Файл:", out_path)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x82 in position 1309: invalid start byte